In [2]:
## IMPORTS
from fileinput import close
from socket import if_nametoindex
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import date, timedelta
from yahoo_fin.stock_info import get_data

In [ ]:
goodSector = pd.DataFrame()
spdf = pd.read_pickle("stockData/Spy.pkl")
sectorOfTicker = pd.read_pickle("stockData/nasdaq.pkl")
sectorOfNyse = pd.read_pickle("stockData/nyse.pkl")
sectorOfTicker.update(sectorOfNyse)

## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

#*Stage Checker
def checkIfStage2(i,price,volumePerc, RS, slope, wMA30,prevStage,prevClose,prevSupport,peak,prevPeak,prevTrough,index,dfSorted,secondBought,initialSupport,fiveYearHigh,param,goodSectorDf):
    if prevStage == "Stage 2" or prevStage == "Buy":
        if spdf.at[index, 'close'] < spdf.at[index, 'WMA30'] * param[13]:
            return "Sell"
        if spdf.at[index, 'WMA30Slope'] < param[14]:
            return "Sell"
        if price>prevPeak:
            dfSorted.iat[i,9] = price
            dfSorted.iat[i,10] = price
        else:
            dfSorted.iat[i,9] = prevPeak
            dfSorted.iat[i,10] = min(prevTrough, price)
        if price < prevSupport*param[7]:
            return "Sell"
        dfSorted.iat[i,11] = prevSupport
        dfSorted.iat[i, 12] = initialSupport
        if price == dfSorted.iat[i,9] and prevTrough < prevPeak*param[6]:
            dfSorted.iat[i,11] = prevTrough
        if secondBought == True:
            dfSorted.at[index, 'secondBuy'] = True
            if price <= initialSupport*param[5] and dfSorted.at[index,'trough'] < dfSorted.at[index,'peak']:
                dfSorted.iat[i, 14] = False        
                return "Buy"
        return "Stage 2"
    try:
        if sectorOfTicker[dfSorted.at[index,'ticker']] in goodSectorDf.at[index,'Sectors']:
            pass
        else:
            return "bad sector"
    except:
        return "bad sector"
    if spdf.at[index, 'close'] < spdf.at[index, 'WMA30'] * param[11]:
        return "bearish"
    if spdf.at[index, 'WMA30Slope'] < param[12]:
        return "bearish"
    if price < fiveYearHigh * param[0]:
        return "resistance"
    if volumePerc < param[1]:
        return "volume"
    if RS < param[2]:
        return "RS"
    if slope < param[3]:
        return "Slope"
    if price < wMA30*param[4]:
        return "Price"
    if price > prevClose*param[8]:
        return "Short"  

    
    indexI = index
    if price>prevClose:
        dfSorted.iat[i,9] = price
        dfSorted.iat[i,10] = price
        dfSorted.iat[i,11] = prevClose
        dfSorted.iat[i,12] = prevClose
    else:
        dfSorted.iat[i,9] = price
        dfSorted.iat[i,10] = price*0.95
        dfSorted.iat[i,11] = price*0.95
        dfSorted.iat[i,12] = price*0.95
    dfSorted.at[index, 'secondBuy'] = True
    return "Buy"





In [4]:

pd.read_pickle("stockData/nyseNasdaq/A.pkl")

,close,volume,ticker,fiveYearHigh,WMA30,WMA30Slope,volumePerc,percent,RS,peak,trough,support,initialSupport,Stage,secondBuy
1999-11-15,28.880545,77780526,A,28.880545,NaN,0.000000,NaN,0.000000,-1.058423,0,0,0,0,,False
1999-11-22,29.461731,19126178,A,29.461731,NaN,0.000000,-0.754101,0.000000,-1.048016,0,0,0,0,,False
1999-11-29,31.831186,17453611,A,31.831186,NaN,0.000000,-0.087449,0.000000,-1.060995,0,0,0,0,,False
1999-12-06,32.010014,10048404,A,32.010014,NaN,0.000000,-0.424279,0.000000,-1.065651,0,0,0,0,,False
1999-12-13,32.859444,15559180,A,32.859444,NaN,0.000000,0.548423,0.000000,-1.059232,0,0,0,0,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13,151.419998,10052100,A,179.279999,158.633485,0.998227,0.348678,1.144520,0.014930,0,0,0,0,,False
2021-12-20,157.800003,6842700,A,179.279999,158.722195,1.000559,-0.319277,1.142402,0.042229,0,0,0,0,,False
2021-12-27,159.649994,4635700,A,179.279999,158.887958,1.001044,-0.322534,1.157723,0.034999,0,0,0,0,,False
2022-01-03,145.149994,10567700,A,179.279999,158.071463,0.994861,1.279634,1.011287,-0.118473,0,0,0,0,,False


In [ ]:
def checkIfStage3(i,price,volumePerc, RS, slope, wMA30,prevStage,prevClose,prevSupport,peak,prevPeak,prevTrough,index,dfSorted,secondBought,initialSupport,fiveYearHigh,param,goodSectorDf):
    if prevStage == "Stage 3" or prevStage == "Sell Short":
        if spdf.at[index, 'close'] > spdf.at[index, 'WMA30'] * a:
            return "Cover"
        if spdf.at[index, 'WMA30Slope'] > b:
            return "Cover"
        if price>prevPeak:
            dfSorted.iat[i,9] = price
            dfSorted.iat[i,10] = price
        else:
            dfSorted.iat[i,9] = prevPeak
            dfSorted.iat[i,10] = min(prevTrough, price)
        if price < prevSupport*param[7]:
            return "Sell"
        dfSorted.iat[i,11] = prevSupport
        dfSorted.iat[i, 12] = initialSupport
        if price == dfSorted.iat[i,9] and prevTrough < prevPeak*param[6]:
            dfSorted.iat[i,11] = prevTrough
        if secondBought == True:
            dfSorted.at[index, 'secondBuy'] = True
            if price <= initialSupport*param[5] and dfSorted.at[index,'trough'] < dfSorted.at[index,'peak']:
                dfSorted.iat[i, 14] = False        
                return "Buy"
        return "Stage 2"
    try:
        if sectorOfTicker[dfSorted.at[index,'ticker']] in goodSectorDf.at[index,'Sectors']:
            pass
        else:
            return "bad sector"
    except:
        return "bad sector"
    if spdf.at[index, 'close'] < spdf.at[index, 'WMA30'] * param[11]:
        return "bearish"
    if spdf.at[index, 'WMA30Slope'] < param[12]:
        return "bearish"
    if price < fiveYearHigh * param[0]:
        return "resistance"
    if volumePerc < param[1]:
        return "volume"
    if RS < param[2]:
        return "RS"
    if slope < param[3]:
        return "Slope"
    if price < wMA30*param[4]:
        return "Price"
    if price > prevClose*param[8]:
        return "Short"  

    
    indexI = index
    if price>prevClose:
        dfSorted.iat[i,9] = price
        dfSorted.iat[i,10] = price
        dfSorted.iat[i,11] = prevClose
        dfSorted.iat[i,12] = prevClose
    else:
        dfSorted.iat[i,9] = price
        dfSorted.iat[i,10] = price*0.95
        dfSorted.iat[i,11] = price*0.95
        dfSorted.iat[i,12] = price*0.95
    dfSorted.at[index, 'secondBuy'] = True
    return "Buy"



In [ ]:
## Main Function
def returnStageDf(dfSorted,param,goodSectorDf):
    first = True
    for index in dfSorted.index:
        if first:
            if dfSorted.index.get_loc(index) == 0:
                first = False
                continue
        prevIndex = index - timedelta(weeks=1)
        i = dfSorted.index.get_loc(index)
        dfSorted.at[index, 'Stage'] = checkIfStage2(i,dfSorted.at[index,'close'],dfSorted.at[index,'volumePerc'],dfSorted.at[index,'RS'],dfSorted.at[index,'WMA30Slope'],dfSorted.at[index,'WMA30'],dfSorted.at[prevIndex,'Stage'],dfSorted.at[prevIndex,'close'],dfSorted.iat[i-1,11],dfSorted.iat[i,9],dfSorted.iat[i-1,9],dfSorted.iat[i-1,10],index,dfSorted,dfSorted.at[prevIndex,'secondBuy'],dfSorted.iat[i-1,12],dfSorted.at[index,'fiveYearHigh'],param,goodSectorDf)
    first = True
    return dfSorted[["close","Stage"]]


In [ ]:
def getStage(ticker,param, goodSectorDf):
#     today = date.today()
#     # #200->1000
    
    # today = today.strftime('%Y-%m-%d')
    # startDate = startDate.strftime('%Y-%m-%d')
    # df = get_data(ticker, start_date=startDate, end_date=today, index_as_date = True, interval="1wk")
    try:
        df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
        return returnStageDf(df,param, goodSectorDf)
    except:
        return pd.DataFrame()

In [ ]:
def getFullDf(ticker,param):
    dfSorted = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
    first = True
    for index in dfSorted.index:
        if first:
            if dfSorted.index.get_loc(index) == 0:
                first = False
                continue
        prevIndex = index - timedelta(weeks=1)
        i = dfSorted.index.get_loc(index)
        dfSorted.at[index, 'Stage'] = checkIfStage2(i,dfSorted.at[index,'close'],dfSorted.at[index,'volumePerc'],dfSorted.at[index,'RS'],dfSorted.at[index,'WMA30Slope'],dfSorted.at[index,'WMA30'],dfSorted.at[prevIndex,'Stage'],dfSorted.at[prevIndex,'close'],dfSorted.iat[i-1,11],dfSorted.iat[i,9],dfSorted.iat[i-1,9],dfSorted.iat[i-1,10],index,dfSorted,dfSorted.at[prevIndex,'secondBuy'],dfSorted.iat[i-1,12],dfSorted.at[index,'fiveYearHigh'],param)
    first = True
    print(len(sectorOfTicker))
    return dfSorted